# Variational Inference

### Estimation of Transition matrix and Emission parameters for each state

In [4]:
import numpy as np
import torch
import pyro
from pyro import poutine
import pyro.distributions as dist
from pyro.infer import SVI, TraceEnum_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam
import pandas as pd
import tqdm

In [5]:
data = pd.read_csv("data/hulls_df_matchday2_reduced.csv")
data = data.dropna()
# Convert the areas from m^2 to dam^2 for computational reasons
data["HomeHull"]=data["HomeHull"]/100
data["AwayHull"]=data["AwayHull"]/100
data.head()

,Time [s],Period,HomeHull,AwayHull
0,2.0,1.0,5.982854,8.106641
1,4.0,1.0,6.470239,9.445797
2,6.0,1.0,6.687160,10.615702
3,8.0,1.0,6.578269,12.365166
4,10.0,1.0,6.843430,14.701596


## Univariate (separate) models

In [ ]:
sequence_home = torch.tensor(data["HomeHull"].values)
sequence_away = torch.tensor(data["AwayHull"].values)

In [ ]:
def model(sequence, hidden_dim, include_prior=True):
    length = len(sequence)
    with poutine.mask(mask=include_prior):
        # Transition probabilities
        probs_x = pyro.sample(
            "probs_x",
            dist.Dirichlet(0.9 * torch.eye(hidden_dim) + 0.1).to_event(),
        )
        # Emission probabilities (1-dimensional for the area)
        probs_alpha = pyro.sample(
            "probs_alpha",
            dist.Gamma(1.0, 1.0).expand([hidden_dim]).to_event(1)
        )

        probs_beta = pyro.sample(
            "probs_beta",
            dist.Gamma(1.0, 1.0).expand([hidden_dim]).to_event(1)
        )
    
    x = 0  # Initial hidden state
    for t in pyro.markov(range(length)):
        x = pyro.sample(
            f"x_{t}",
            dist.Categorical(probs_x[x]),
            infer={"enumerate": "parallel"},
        )
        pyro.sample(
            f"y_{t}",
            dist.Gamma(probs_alpha[x], probs_beta[x]),
            obs=sequence[t],
        )
# Define the guide (variational distribution)
guide = AutoDelta(poutine.block(model, expose=["probs_x", "probs_alpha", "probs_beta"]))

---

**⚠️ DANGER ZONE ⚠️**

In [ ]:
pyro.clear_param_store()

---

In [ ]:
# Optimizer
optimizer = Adam({"lr": 0.01})

# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(model, guide, optimizer, loss=elbo)

# Training
num_steps = 1000
hidden_dim = 2
tqdm_bar = tqdm.tqdm(range(num_steps))
for step in tqdm_bar:
    loss = svi.step(sequence_home, hidden_dim)
    if step % 100 == 0:
         tqdm_bar.set_postfix({'LOSS': loss})

In [ ]:
posterior = guide(sequence_home,hidden_dim)
# can i save posterior?
posterior

In [ ]:
torch.save(posterior,"parameters/univariateHMM_matchday2_HomeTeam_2states.pt")

...Here print means of each state...

## Bivariate copula model

In [8]:
xy_sequence = torch.tensor(data[["HomeHull","AwayHull"]].values) #we divide by 100 to avoid computational issues (insted of m^2 we use dm^2)
xy_sequence.shape

torch.Size([1970, 2])

In [9]:
def empirical_gamma_cdf(x, shape, rate):
    # Generate 1000 random samples from the gamma distribution
    # Think about setting a seed to avoid too much stochasticity
    #torch.manual_seed(3407)
    samples = torch.distributions.Gamma(shape, rate).sample((3500,))
    return (samples <= x).float().mean()

def copula_term_log(theta: torch.tensor, u: torch.tensor, v: torch.tensor):
    log_numerator = torch.log(theta) + torch.log(torch.exp(theta) - 1.0) + theta * (1.0 + u + v)
    denominator = (torch.exp(theta) - torch.exp(theta + theta * u) + torch.exp(theta * (u + v)) - torch.exp(theta + theta * v))**2
    log_denominator = torch.log(denominator)
    return log_numerator - log_denominator

def copulamodel_log_pdf(x,y,shape1,rate1,shape2,rate2,theta):
    g1_lpdf= dist.Gamma(shape1,rate1).log_prob(x)
    g2_lpdf= dist.Gamma(shape2,rate2).log_prob(y)
    u= empirical_gamma_cdf(x, shape1, rate1)
    v= empirical_gamma_cdf(y, shape2, rate2)
    # Qui pensare se fare una if su u e v diversi da circa 0...in quel caso non calcolare il copula term (lo lascio nullo)
    lpdf=g1_lpdf+g2_lpdf
    if (torch.abs(u) > 1e-6) & (torch.abs(v) > 1e-6):
        lpdf += copula_term_log(theta=theta,u=u,v=v)
    return lpdf
    

In [25]:
def model(sequence, hidden_dim, include_prior=True):
    n_obs = sequence.shape[0]
    with poutine.mask(mask=include_prior):
        #---------------------------------------------------------------------
        # Transition probabilities
        probs_x = pyro.sample(
            "probs_x",
            dist.Dirichlet(0.9 * torch.eye(hidden_dim) + 0.1).to_event(),
        )
        #---------------------------------------------------------------------
        # Prior for the parameters of emission probabilities 
        probs_alpha1 = pyro.sample(
            "probs_alpha1",
            dist.Gamma(concentration=15.0,rate=0.8).expand([hidden_dim]).to_event(1)
        )

        probs_beta1 = pyro.sample(
            "probs_beta1",
            dist.Gamma(concentration=1.0,rate=1.0).expand([hidden_dim]).to_event(1)
        )
        probs_alpha2 = pyro.sample(
            "probs_alpha2",
            dist.Gamma(concentration=15.0,rate=0.8).expand([hidden_dim]).to_event(1)
        )

        probs_beta2 = pyro.sample(
            "probs_beta2",
            dist.Gamma(concentration=1.0,rate=1.0).expand([hidden_dim]).to_event(1)
        )
        #---------------------------------------------------------------------
        # Prior for theta
        theta = pyro.sample(
            "theta",
            dist.Gamma(5.0,0.7).expand([hidden_dim]).to_event(1)
        )
        
    
    x = 0  # Initial hidden state
    for t in pyro.markov(range(n_obs)):
        x = pyro.sample(
            f"x_{t}",
            dist.Categorical(probs_x[x]),
            infer={"enumerate": "parallel"},
        )
        log_pdf = copulamodel_log_pdf(
            x=sequence[t,0],
            y=sequence[t,1],
            shape1=probs_alpha1[x],
            rate1=probs_beta1[x],
            shape2=probs_alpha2[x],
            rate2=probs_beta2[x],
            theta=theta[x]
        )
        pyro.factor(f"xy_{t}", log_pdf)
        

guide = AutoDelta(poutine.block(model, expose=["probs_x",
                                               "probs_alpha1",
                                               "probs_beta1",
                                               "probs_alpha2",
                                               "probs_beta2",
                                               "theta"
                                               ]))

In [24]:
pyro.clear_param_store()

In [26]:
# Optimizer
optimizer = Adam({"lr": 0.01})

# Inference algorithm
elbo = TraceEnum_ELBO(max_plate_nesting=1)
svi = SVI(model, guide, optimizer, loss=elbo)

# Training
hidden_dim=5
num_steps = 400
tqdm_bar = tqdm.tqdm(range(num_steps))
for step in tqdm_bar:
    loss = svi.step(xy_sequence, hidden_dim)
    #if step % 50 == 0:
    tqdm_bar.set_postfix({'LOSS': loss})

100%|██████████| 400/400 [22:24<00:00,  3.36s/it, LOSS=8.64e+3]


In [27]:
posterior = guide(xy_sequence,hidden_dim)
posterior

{'probs_x': tensor([[0.8495, 0.0779, 0.0116, 0.0279, 0.0330],
         [0.1199, 0.4459, 0.1262, 0.1403, 0.1677],
         [0.0067, 0.0287, 0.8834, 0.0501, 0.0311],
         [0.0059, 0.0335, 0.0142, 0.9367, 0.0097],
         [0.0247, 0.0239, 0.0349, 0.0175, 0.8989]], grad_fn=<ExpandBackward0>),
 'probs_alpha1': tensor([ 6.3711, 11.6862, 15.3060, 24.0355, 23.1379],
        grad_fn=<ExpandBackward0>),
 'probs_beta1': tensor([1.4893, 1.5593, 1.4200, 2.2907, 3.6762], grad_fn=<ExpandBackward0>),
 'probs_alpha2': tensor([ 8.6080, 12.1319, 19.0160, 11.6785, 39.8159],
        grad_fn=<ExpandBackward0>),
 'probs_beta2': tensor([1.2431, 1.3855, 1.4573, 1.7556, 3.3303], grad_fn=<ExpandBackward0>),
 'theta': tensor([2.5947, 7.3676, 4.0668, 0.5516, 0.7350], grad_fn=<ExpandBackward0>)}

In [28]:
meanH_state0 = posterior["probs_alpha1"][0]/posterior["probs_beta1"][0]*100
meanA_state0 = posterior["probs_alpha2"][0]/posterior["probs_beta2"][0]*100
meanH_state1 = posterior["probs_alpha1"][1]/posterior["probs_beta1"][1]*100
meanA_state1 = posterior["probs_alpha2"][1]/posterior["probs_beta2"][1]*100
meanH_state2 = posterior["probs_alpha1"][2]/posterior["probs_beta1"][2]*100
meanA_state2 = posterior["probs_alpha2"][2]/posterior["probs_beta2"][2]*100
meanH_state3 = posterior["probs_alpha1"][3]/posterior["probs_beta1"][3]*100
meanA_state3 = posterior["probs_alpha2"][3]/posterior["probs_beta2"][3]*100
meanH_state4 = posterior["probs_alpha1"][4]/posterior["probs_beta1"][4]*100
meanA_state4 = posterior["probs_alpha2"][4]/posterior["probs_beta2"][4]*100


print(f">> Mean of the Convex Hull for home team (STATE 0): {meanH_state0:.2f} m^2")
print(f">> Mean of the Convex Hull for away team (STATE 0): {meanA_state0:.2f} m^2")
print(f">> Mean of the Convex Hull for home team (STATE 1): {meanH_state1:.2f} m^2")
print(f">> Mean of the Convex Hull for away team (STATE 1): {meanA_state1:.2f} m^2")
print(f">> Mean of the Convex Hull for home team (STATE 2): {meanH_state2:.2f} m^2")
print(f">> Mean of the Convex Hull for away team (STATE 2): {meanA_state2:.2f} m^2")
print(f">> Mean of the Convex Hull for home team (STATE 3): {meanH_state3:.2f} m^2")
print(f">> Mean of the Convex Hull for away team (STATE 3): {meanA_state3:.2f} m^2")
print(f">> Mean of the Convex Hull for home team (STATE 4): {meanH_state4:.2f} m^2")
print(f">> Mean of the Convex Hull for away team (STATE 4): {meanA_state4:.2f} m^2")


>> Mean of the Convex Hull for home team (STATE 0): 427.80 m^2
>> Mean of the Convex Hull for away team (STATE 0): 692.47 m^2
>> Mean of the Convex Hull for home team (STATE 1): 749.43 m^2
>> Mean of the Convex Hull for away team (STATE 1): 875.65 m^2
>> Mean of the Convex Hull for home team (STATE 2): 1077.92 m^2
>> Mean of the Convex Hull for away team (STATE 2): 1304.89 m^2
>> Mean of the Convex Hull for home team (STATE 3): 1049.28 m^2
>> Mean of the Convex Hull for away team (STATE 3): 665.20 m^2
>> Mean of the Convex Hull for home team (STATE 4): 629.39 m^2
>> Mean of the Convex Hull for away team (STATE 4): 1195.58 m^2


In [29]:
torch.save(posterior,"parameters/copulaHMM_matchday2_5states.pt")